In [ ]:
# 경로에 존재하는 데이터를 읽어오기 위한 모듈
import pandas as pd

# 주어진 데이터 셋 내에서 일정 비율 아래, 훈련 / 검증 셋으로의 분리를 위한 모듈
from sklearn.model_selection import train_test_split

# 데이터 셋 내 존재하는 문자열 구성 데이터를 정량적 데이터로 변환하기 위한 인코더와 데이터 분포를 조정할 스케일러
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [ ]:
'''
해당 함수는 데이터 셋와 인코딩이 필요한 문자열 리스트, 목표 문자열을 입력 받아, 인코딩된 데이터 셋과 목표 셋으로 분리합니다
문자열로 구성된 데이터 중 결과 값의 예측과 상관성이 있는 것을 추려내기 위함입니다.

DATASET = pd.DataFrame 의 자료형을 입력 받습니다.
encoding_col = 문자열의 리스트로 인코딩이 필요한 열의 이름들이 포함 되어있습니다.
target = 목표 열의 이름을 문자열로 받습니다.
return = 인코딩 된 열이 포함된 DataFrame 과 분리된 목표 열을 반환합니다.
'''
def encode_sep(DATASET, encoding_col, target):
  # 인코더를 호출하고, 목표열을 인코딩 합니다.
  L_encoder = LabelEncoder()
  DATASET[target] = L_encoder.fit_transform(DATASET[target])

  # 입력 받은 문자열 리스트 내의 열을 인코딩 합니다.
  for col_name in encoding_col: 
    DATASET[col_name] = L_encoder.fit_transform(DATASET[col_name])
    
  # 데이터 셋의 목표 열을 지정, 저장하고 데이터 셋 내에서 분리합니다.
  labeled = DATASET[target]
  DATASET = DATASET.drop(target, axis = 1)
  
  return DATASET, labeled

'''
해당 함수는 데이터 셋과 목표열 임계값을 인자로 받아, 목표 열과 데이터 셋 내 나머지의 열 간의 상관계수를 절대값으로 측정하고
해당 절대값이 임계값 이하 이면 제거 하는 역할을 수행하고 제거 된 데이터 셋을 반환합니다.
이는 각 열이 특성을 나타냄으로서, 목표 열의 결정 사유를 예측하는 것에 대한 영향이 적은 것을 제거해 보다 정확한 예측을 위함입니다.

DATASET = pd.DataFrame 의 자료형을 입력 받습니다.
target = 목표 열의 이름을 문자열로 받습니다.
threshold = 넘지 못한다면 제거 해야할 임계값을 정수형으로 받습니다.
'''

def correlation_wise(DATASET, target, threshold):
  # 목표 열과 데이터 셋 내 다른 열과의 데이터 상관성을 절대값을 적용하여 반환합니다.
  correlation_matrix = DATASET.corrwith(DATASET[target], method='spearman').abs()

  # 상관계수 표로 부터 인덱스와 값을 동시에 불러와, 값이 지정된 임계 값보다 작을 경우 제거하는 역할을 수행합니다.
  for index, value in enumerate(correlation_matrix.values):
    if value < threshold : 
      DATASET = DATASET.drop([correlation_matrix.index[index]], axis= 1)

  return DATASET

'''
해당 함수는 데이터의 경로를 입력 받아, 데이터 셋을 pd.DataFrame 형태로 생성하고 
결측치가 존재하는 행, 값이 존재치 않는 열을 제거합니다.
위의 정의 된 상관계수 누락 함수와, 인코딩 함수 또한 실행이 된후, 데이터 셋과 목표 값 리스트를 반환합니다.

이는 예측에 있어서의 잡음을 최소화 하고, 더 좋은 성능을 내기 위함입니다.
'''

def preprocessor(DATASET_PATH, encoding_col, target, threshold = 0.005):

  # 경로 상의 .csv 파일을 변수 이름하의 메모리에 불러오고, 결측치 NaN 값을 포함하는 행을 제거합니다.
  DATASET = pd.read_csv(DATASET_PATH)
  DATASET = DATASET.dropna(axis = 0)

  # 데이터셋의 열 이름들을 리스트의 형태로 저장합니다.
  COLs = list(DATASET.columns)

  # 리스트 내 열 이름을 따라 각 열의 데이터 총합이 0인, 즉 열 내에 값이 없는 열을 제거합니다.
  for column in COLs: 
    if not DATASET[column].sum(): 
      DATASET = DATASET.drop(column, axis = 1)

  # 위에서 정의한 상관관계 기반 열 선택 제거 함수를 호출 합니다.    
  DATASET = correlation_wise(DATASET, target, threshold = threshold)

  # 위에서 정의한 인코딩, 결과값 분리 함수를 호출 합니다.
  DATASET, label = encode_sep(DATASET, encoding_col, target)
  
  return DATASET, label

'''
해당 함수는 모델의 입력 셋과 결과 셋을 지정된 비율로 훈련 셋과 검증 셋으로 분리하고, 입력 셋 내 값들을 일정 분포로 조정합니다.

feature = 데이터 셋 내의 값들을 추출한 2차원 행렬입니다. 행 = 데이터, 열 = 특성
label = 위의 전처리 함수를 통해 분리되어 나온 결과 값을 추출한 단일 차원 행렬입니다.
valid_size = 데이터 셋 내의 훈련 셋과 검증 셋의 비율로 ex) 0.4 일 경우 훈련 : 검증 = 0.6 : 0.4 로 분할됩니다.
randon_state = 매번의 분할은 임의의 값 분포로 결정됩니다, ex) 결과가 1, 2, 3 의 클래스 인 입력치의 분포가 매번 다릅니다.
이를 고정시켜 최대한 값의 관측에 지장이 없도록 시드 번호를 부여합니다.
'''

def Split_n_Scale(feature, label, valid_size, random_state = 25):
  # 이상치에도 민감치 않게 넓은 분포로 데이터를 조정하기 위해 RobustScaler를 사용했습니다.
  scaler = RobustScaler()
  
  # 데이터 셋을 분리합니다.
  x_train, x_valid, y_train, y_valid = train_test_split(feature, label, test_size= valid_size, random_state= random_state)
  
  # 위에서 선언한 Scaler로 데이터의 분포를 조정합니다.
  x_train = scaler.fit_transform(x_train)
  x_valid = scaler.transform(x_valid)

  return x_train, x_valid, y_train, y_valid, scaler